# 🏥 Hospital Patient Records – Comprehensive ML Analysis

**Dataset:** Synthea-generated records from Massachusetts General Hospital
| Table | Rows | Key Fields |
|---|---|---|
| Patients | 974 | Demographics, birth/death, location |
| Encounters | 27,891 | Visit type, costs, diagnosis, payer |
| Procedures | 47,701 | Medical procedures, costs |
| Payers | 10 | Insurance providers |

**ML Tasks:** Encounter Classification · Mortality Prediction · Cost Regression · Patient Clustering
**Techniques:** 10+ algorithms, cross-validation, hyperparameter tuning, ensemble methods, feature importance

In [1]:
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, os, base64
from io import BytesIO

from sklearn.model_selection import (train_test_split, cross_val_score,
    StratifiedKFold, GridSearchCV, RandomizedSearchCV, learning_curve)
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix, classification_report, roc_curve, auc,
    mean_squared_error, mean_absolute_error, r2_score, silhouette_score)
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (RandomForestClassifier, GradientBoostingClassifier,
    RandomForestRegressor, GradientBoostingRegressor,
    VotingClassifier, StackingClassifier, AdaBoostClassifier)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA

try:
    from jinja2 import Template
    HAS_JINJA2 = True
except ImportError:
    HAS_JINJA2 = False

warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
plt.rcParams["figure.dpi"] = 100
plt.rcParams["font.size"] = 11

SEED = 42
np.random.seed(SEED)
os.makedirs("outputs/plots", exist_ok=True)

report_images = {}

def save_plot(fig, name):
    """Save a figure to PNG and store base64 for the report."""
    fig.savefig(f"outputs/plots/{name}.png", dpi=150, bbox_inches="tight")
    buf = BytesIO()
    fig.savefig(buf, format="png", dpi=150, bbox_inches="tight")
    buf.seek(0)
    report_images[name] = base64.b64encode(buf.read()).decode("utf-8")
    plt.close(fig)
    print(f"  Plot saved: {name}.png")

# Result dictionaries
results = {}
regression_results = {}
mortality_results = {}

print("All imports successful")
print(f"  scikit-learn {__import__('sklearn').__version__}")
print(f"  pandas {pd.__version__}, numpy {np.__version__}")
print(f"  Jinja2: {HAS_JINJA2}")

All imports successful
  scikit-learn 1.8.0
  pandas 3.0.1, numpy 2.4.2
  Jinja2: True


## 1. Data Loading & Exploration

In [2]:
DATA_DIR = "Hospital+Patient+Records"
patients    = pd.read_csv(f"{DATA_DIR}/patients.csv")
encounters  = pd.read_csv(f"{DATA_DIR}/encounters.csv")
procedures  = pd.read_csv(f"{DATA_DIR}/procedures.csv")
organizations = pd.read_csv(f"{DATA_DIR}/organizations.csv")
payers      = pd.read_csv(f"{DATA_DIR}/payers.csv")

print("Dataset Loaded:")
for name, df in [("patients", patients), ("encounters", encounters),
                  ("procedures", procedures), ("organizations", organizations),
                  ("payers", payers)]:
    print(f"  {name:15s}: {df.shape[0]:>6,} rows x {df.shape[1]:>2} columns")

patients.head()

Dataset Loaded:
  patients       :    974 rows x 20 columns
  encounters     : 27,891 rows x 14 columns
  procedures     : 47,701 rows x  9 columns
  organizations  :      1 rows x  8 columns
  payers         :     10 rows x  7 columns


,Id,BIRTHDATE,DEATHDATE,PREFIX,FIRST,LAST,SUFFIX,MAIDEN,MARITAL,RACE,ETHNICITY,GENDER,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON
0,5605b66b-e92d-c16c-1b83-b8bf7040d51f,1977-03-19,NaN,Mrs.,Nikita578,Erdman779,NaN,Leannon79,M,white,nonhispanic,F,Wakefield Massachusetts US,510 Little Station Unit 69,Quincy,Massachusetts,Norfolk County,2186.0,42.290937,-70.975503
1,6e5ae27c-8038-7988-e2c0-25a103f01bfa,1940-02-19,NaN,Mr.,Zane918,Hodkiewicz467,NaN,NaN,M,white,nonhispanic,M,Brookline Massachusetts US,747 Conn Throughway,Boston,Massachusetts,Suffolk County,2135.0,42.308831,-71.063162
2,8123d076-0886-9007-e956-d5864aa121a7,1958-06-04,NaN,Mr.,Quinn173,Marquardt819,NaN,NaN,M,white,nonhispanic,M,Gardner Massachusetts US,816 Okuneva Extension Apt 91,Quincy,Massachusetts,Norfolk County,2170.0,42.265177,-70.967085
3,770518e4-6133-648e-60c9-071eb2f0e2ce,1928-12-25,2017-09-29,Mr.,Abel832,Smitham825,NaN,NaN,M,white,hispanic,M,Randolph Massachusetts US,127 Cole Way Unit 95,Boston,Massachusetts,Suffolk County,2118.0,42.334304,-71.066801
4,f96addf5-81b9-0aab-7855-d208d3d352c5,1928-12-25,2014-02-23,Mr.,Edwin773,Labadie908,NaN,NaN,M,white,hispanic,M,Stow Massachusetts US,976 Ziemann Gateway,Boston,Massachusetts,Suffolk County,2125.0,42.346771,-71.058813


In [3]:
print("=" * 60)
print("PATIENTS TABLE")
print("=" * 60)
missing = patients.isnull().sum()
missing = missing[missing > 0]
print(f"Missing values:\n{missing}" if len(missing) else "No missing values (required fields)")
print(f"\nGender:    {patients['GENDER'].value_counts().to_dict()}")
print(f"Race:      {patients['RACE'].value_counts().to_dict()}")
print(f"Marital:   {patients['MARITAL'].value_counts().to_dict()}")
deceased_n = patients["DEATHDATE"].notna().sum()
print(f"Deceased:  {deceased_n} / {len(patients)} ({deceased_n/len(patients)*100:.1f}%)")

print("\n" + "=" * 60)
print("ENCOUNTERS TABLE")
print("=" * 60)
print(f"Classes:  {encounters['ENCOUNTERCLASS'].value_counts().to_dict()}")
print(f"Date range: {encounters['START'].min()[:10]} to {encounters['START'].max()[:10]}")
print(encounters[["BASE_ENCOUNTER_COST","TOTAL_CLAIM_COST","PAYER_COVERAGE"]].describe().round(2))

print("\n" + "=" * 60)
print("TOP 10 PROCEDURES")
print("=" * 60)
print(procedures["DESCRIPTION"].value_counts().head(10))
print("\nPAYERS:")
print(payers[["NAME"]].to_string())

PATIENTS TABLE
Missing values:
DEATHDATE    820
SUFFIX       953
MAIDEN       588
MARITAL        1
ZIP          142
dtype: int64

Gender:    {'M': 494, 'F': 480}
Race:      {'white': 680, 'black': 163, 'asian': 91, 'other': 16, 'hawaiian': 13, 'native': 11}
Marital:   {'M': 784, 'S': 189}
Deceased:  154 / 974 (15.8%)

ENCOUNTERS TABLE
Classes:  {'ambulatory': 12537, 'outpatient': 6300, 'urgentcare': 3666, 'emergency': 2322, 'wellness': 1931, 'inpatient': 1135}
Date range: 2011-01-02 to 2022-02-05
       BASE_ENCOUNTER_COST  TOTAL_CLAIM_COST  PAYER_COVERAGE
count             27891.00          27891.00        27891.00
mean                116.18           3639.68         1114.97
std                  28.41           9205.60         4768.62
min                  85.55              0.00            0.00
25%                  85.55            142.58            0.00
50%                 136.80            278.58           28.44
75%                 142.58           1412.53          155.77
max       

## 2. Data Integration & Feature Engineering

Merge all five tables and create derived features for ML modelling.

In [5]:
# --- Parse dates ---
patients["BIRTHDATE"] = pd.to_datetime(patients["BIRTHDATE"])
patients["DEATHDATE"] = pd.to_datetime(patients["DEATHDATE"])
patients["Is_Deceased"] = patients["DEATHDATE"].notna().astype(int)
encounters["START"] = pd.to_datetime(encounters["START"], utc=True)
encounters["STOP"]  = pd.to_datetime(encounters["STOP"], utc=True)

ref_date = encounters["START"].max()
patients["BIRTHDATE"] = patients["BIRTHDATE"].dt.tz_localize("UTC")
patients["DEATHDATE"] = patients["DEATHDATE"].dt.tz_localize("UTC")
patients["Age"] = ((ref_date - patients["BIRTHDATE"]).dt.days / 365.25).astype(int)

# --- Aggregate procedures per encounter ---
proc_agg = procedures.groupby("ENCOUNTER").agg(
    num_procedures=("CODE", "count"),
    total_proc_cost=("BASE_COST", "sum"),
    avg_proc_cost=("BASE_COST", "mean"),
    unique_procs=("CODE", "nunique"),
).reset_index()

# --- Merge encounters <- patients ---
enc = encounters.merge(
    patients[["Id","BIRTHDATE","GENDER","RACE","ETHNICITY","MARITAL","Age","Is_Deceased"]],
    left_on="PATIENT", right_on="Id", how="left", suffixes=("","_p")
).drop(columns=["Id_p"], errors="ignore")

# --- Merge <- payers ---
enc = enc.merge(
    payers[["Id","NAME"]].rename(columns={"NAME": "Payer_Name"}),
    left_on="PAYER", right_on="Id", how="left", suffixes=("","_pay")
).drop(columns=["Id_pay"], errors="ignore")

# --- Merge <- procedures aggregation ---
enc = enc.merge(proc_agg, left_on="Id", right_on="ENCOUNTER", how="left")
for c in ["num_procedures","unique_procs"]:
    enc[c] = enc[c].fillna(0).astype(int)
for c in ["total_proc_cost","avg_proc_cost"]:
    enc[c] = enc[c].fillna(0.0)

# --- Temporal features ---
enc["Duration_hours"] = (enc["STOP"] - enc["START"]).dt.total_seconds() / 3600
enc["Hour"]      = enc["START"].dt.hour
enc["DayOfWeek"] = enc["START"].dt.dayofweek
enc["Month"]     = enc["START"].dt.month
enc["Year"]      = enc["START"].dt.year
enc["Is_Weekend"] = (enc["DayOfWeek"] >= 5).astype(int)

# --- Cost features ---
enc["Out_of_pocket"]  = enc["TOTAL_CLAIM_COST"] - enc["PAYER_COVERAGE"]
enc["Coverage_ratio"] = np.where(
    enc["TOTAL_CLAIM_COST"] > 0,
    enc["PAYER_COVERAGE"] / enc["TOTAL_CLAIM_COST"], 0)
enc["Has_reason"] = enc["REASONCODE"].notna().astype(int)
enc["Log_cost"]   = np.log1p(enc["TOTAL_CLAIM_COST"])

# --- Encode categoricals ---
le_enc_class = LabelEncoder()
enc["EncClass_enc"] = le_enc_class.fit_transform(enc["ENCOUNTERCLASS"])
encounter_names = le_enc_class.classes_

for col in ["GENDER","RACE","ETHNICITY"]:
    enc[f"{col}_enc"] = LabelEncoder().fit_transform(enc[col])
enc["MARITAL_enc"] = LabelEncoder().fit_transform(enc["MARITAL"].fillna("U"))
enc["Payer_enc"]   = LabelEncoder().fit_transform(enc["Payer_Name"].fillna("Unknown"))

# --- Patient-level aggregation ---
pat = enc.groupby("PATIENT").agg(
    total_encounters=("Id","count"),
    total_cost=("TOTAL_CLAIM_COST","sum"),
    avg_cost=("TOTAL_CLAIM_COST","mean"),
    max_cost=("TOTAL_CLAIM_COST","max"),
    total_procs=("num_procedures","sum"),
    avg_procs=("num_procedures","mean"),
    emergency_n=("ENCOUNTERCLASS", lambda x: (x=="emergency").sum()),
    inpatient_n=("ENCOUNTERCLASS", lambda x: (x=="inpatient").sum()),
    ambulatory_n=("ENCOUNTERCLASS", lambda x: (x=="ambulatory").sum()),
    wellness_n=("ENCOUNTERCLASS", lambda x: (x=="wellness").sum()),
    n_enc_types=("ENCOUNTERCLASS","nunique"),
    total_coverage=("PAYER_COVERAGE","sum"),
    avg_duration=("Duration_hours","mean"),
    reason_n=("Has_reason","sum"),
).reset_index()

pat = pat.merge(
    patients[["Id","Age","GENDER","RACE","ETHNICITY","MARITAL","Is_Deceased"]],
    left_on="PATIENT", right_on="Id", how="left")
pat["Gender_enc"]    = LabelEncoder().fit_transform(pat["GENDER"])
pat["Race_enc"]      = LabelEncoder().fit_transform(pat["RACE"])
pat["Ethnicity_enc"] = LabelEncoder().fit_transform(pat["ETHNICITY"])
pat["Marital_enc"]   = LabelEncoder().fit_transform(pat["MARITAL"].fillna("U"))
pat["Emergency_ratio"] = pat["emergency_n"] / pat["total_encounters"]
pat["Cost_per_enc"]    = pat["total_cost"] / pat["total_encounters"]

print(f"Encounter-level dataset: {enc.shape}")
print(f"Patient-level dataset:   {pat.shape}")
print(f"\nEncounter classes:\n{enc['ENCOUNTERCLASS'].value_counts()}")
print(f"\nMortality: {pat['Is_Deceased'].value_counts().to_dict()}")

enc.to_csv("outputs/encounters_merged.csv", index=False)
pat.to_csv("outputs/patients_aggregated.csv", index=False)
print("\nMerged datasets saved to outputs/")

Encounter-level dataset: (27891, 43)
Patient-level dataset:   (974, 28)

Encounter classes:
ENCOUNTERCLASS
ambulatory    12537
outpatient     6300
urgentcare     3666
emergency      2322
wellness       1931
inpatient      1135
Name: count, dtype: int64

Mortality: {0: 820, 1: 154}

Merged datasets saved to outputs/


## 3. Exploratory Data Analysis

In [6]:
# ---- Figure 1: Overview dashboard (2x2) ----
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1a  Encounter type distribution
ec = enc["ENCOUNTERCLASS"].value_counts()
colors = sns.color_palette("Set2", len(ec))
bars = axes[0,0].bar(ec.index, ec.values, color=colors)
axes[0,0].set_title("Encounter Type Distribution", fontweight="bold")
axes[0,0].set_ylabel("Count")
axes[0,0].tick_params(axis="x", rotation=45)
for b, v in zip(bars, ec.values):
    axes[0,0].text(b.get_x()+b.get_width()/2, b.get_height()+200,
                   f"{v:,}", ha="center", fontsize=9)

# 1b  Patient race
demo = patients["RACE"].value_counts()
axes[0,1].pie(demo.values, labels=demo.index, autopct="%1.1f%%",
              colors=sns.color_palette("pastel"), startangle=140)
axes[0,1].set_title("Patient Race Distribution", fontweight="bold")

# 1c  Cost box-plot
sns.boxplot(data=enc, x="ENCOUNTERCLASS", y="TOTAL_CLAIM_COST",
            ax=axes[1,0], palette="Set2", showfliers=False)
axes[1,0].set_title("Cost by Encounter Type", fontweight="bold")
axes[1,0].set_ylabel("Total Claim Cost ($)")
axes[1,0].tick_params(axis="x", rotation=45)

# 1d  Age by mortality
for lbl, grp in pat.groupby("Is_Deceased"):
    name = "Deceased" if lbl else "Alive"
    axes[1,1].hist(grp["Age"], bins=25, alpha=0.6, label=name, density=True)
axes[1,1].set_title("Age Distribution by Mortality", fontweight="bold")
axes[1,1].set_xlabel("Age"); axes[1,1].set_ylabel("Density")
axes[1,1].legend()
fig.tight_layout()
save_plot(fig, "overview_dashboard")

# ---- Figure 2: Encounters over time ----
fig, ax = plt.subplots(figsize=(14, 5))
monthly = enc.set_index("START").resample("ME").size()
ax.plot(monthly.index, monthly.values, color="steelblue", linewidth=1.5)
ax.fill_between(monthly.index, monthly.values, alpha=0.15, color="steelblue")
ax.set_title("Monthly Encounter Volume", fontweight="bold")
ax.set_xlabel("Date"); ax.set_ylabel("Encounters")
fig.tight_layout()
save_plot(fig, "encounters_timeline")

# ---- Figure 3: Correlation heatmap ----
num_cols = ["Age","Duration_hours","BASE_ENCOUNTER_COST","TOTAL_CLAIM_COST",
            "PAYER_COVERAGE","Out_of_pocket","Coverage_ratio",
            "num_procedures","total_proc_cost","Hour","Month"]
corr = enc[num_cols].corr()
fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap="coolwarm",
            center=0, ax=ax, square=True, linewidths=0.5)
ax.set_title("Feature Correlation Heatmap", fontweight="bold")
fig.tight_layout()
save_plot(fig, "correlation_heatmap")

# ---- Figure 4: Payer distribution ----
fig, ax = plt.subplots(figsize=(12, 5))
pc = enc["Payer_Name"].value_counts()
bars = ax.barh(pc.index, pc.values, color=sns.color_palette("viridis", len(pc)))
ax.set_title("Encounters by Insurance Payer", fontweight="bold")
ax.set_xlabel("Count")
for b, v in zip(bars, pc.values):
    ax.text(b.get_width()+50, b.get_y()+b.get_height()/2,
            f"{v:,}", va="center", fontsize=9)
fig.tight_layout()
save_plot(fig, "payer_distribution")

# ---- Figure 5: Top procedures ----
fig, ax = plt.subplots(figsize=(12, 6))
tp = procedures["DESCRIPTION"].value_counts().head(15)
ax.barh(tp.index[::-1], tp.values[::-1], color="steelblue")
ax.set_title("Top 15 Medical Procedures", fontweight="bold")
ax.set_xlabel("Count")
fig.tight_layout()
save_plot(fig, "top_procedures")

# ---- Figure 6: Gender / encounter heatmap ----
fig, ax = plt.subplots(figsize=(10, 4))
ct = pd.crosstab(enc["GENDER"], enc["ENCOUNTERCLASS"], normalize="index")
sns.heatmap(ct, annot=True, fmt=".3f", cmap="YlOrRd", ax=ax)
ax.set_title("Encounter Type Proportion by Gender", fontweight="bold")
fig.tight_layout()
save_plot(fig, "gender_encounter_heatmap")

print("All EDA visualisations saved!")

  Plot saved: overview_dashboard.png
  Plot saved: encounters_timeline.png
  Plot saved: correlation_heatmap.png
  Plot saved: payer_distribution.png
  Plot saved: top_procedures.png
  Plot saved: gender_encounter_heatmap.png
All EDA visualisations saved!


## 4. Encounter-Type Classification

Predict the type of hospital encounter (ambulatory, emergency, inpatient, outpatient,
urgentcare, wellness) using patient demographics, temporal features, and procedure data.

In [7]:
enc_features = ["Age","GENDER_enc","RACE_enc","ETHNICITY_enc","MARITAL_enc",
                "Payer_enc","Hour","DayOfWeek","Month","Is_Weekend",
                "Has_reason","num_procedures","total_proc_cost"]

X = enc[enc_features].values
y = enc["EncClass_enc"].values

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y)
scaler = StandardScaler()
X_tr_s = scaler.fit_transform(X_tr)
X_te_s = scaler.transform(X_te)

print(f"Train: {X_tr.shape[0]:,}  |  Test: {X_te.shape[0]:,}")
print(f"Features: {len(enc_features)}")
print(f"Classes ({len(encounter_names)}): {list(encounter_names)}")

def evaluate_clf(name, model, Xtr, Xte, ytr, yte, store=results):
    """Train, predict, evaluate, and store a classifier."""
    model.fit(Xtr, ytr)
    yp = model.predict(Xte)
    acc  = accuracy_score(yte, yp)
    prec = precision_score(yte, yp, average="weighted", zero_division=0)
    rec  = recall_score(yte, yp, average="weighted", zero_division=0)
    f1   = f1_score(yte, yp, average="weighted", zero_division=0)
    store[name] = {"accuracy": acc, "precision": prec, "recall": rec,
                   "f1_score": f1, "y_pred": yp, "model": model}
    print(f"  {name:28s}  Acc={acc:.4f}  Prec={prec:.4f}  Rec={rec:.4f}  F1={f1:.4f}")
    return model, yp

Train: 22,312  |  Test: 5,579
Features: 13
Classes (6): ['ambulatory', 'emergency', 'inpatient', 'outpatient', 'urgentcare', 'wellness']


In [ ]:
print("Training Encounter-Type Classifiers")
print("=" * 90)

# 1. Logistic Regression
evaluate_clf("Logistic Regression",
    LogisticRegression(max_iter=1000, random_state=SEED),
    X_tr_s, X_te_s, y_tr, y_te)

# 2. Decision Tree
dt, _ = evaluate_clf("Decision Tree",
    DecisionTreeClassifier(max_depth=15, random_state=SEED),
    X_tr, X_te, y_tr, y_te)
dt_imp = pd.Series(dt.feature_importances_, index=enc_features)

# 3. Random Forest
rf, _ = evaluate_clf("Random Forest",
    RandomForestClassifier(n_estimators=200, max_depth=20,
                           random_state=SEED, n_jobs=-1),
    X_tr, X_te, y_tr, y_te)
rf_imp = pd.Series(rf.feature_importances_, index=enc_features)

# 4. Gradient Boosting
gb, _ = evaluate_clf("Gradient Boosting",
    GradientBoostingClassifier(n_estimators=200, learning_rate=0.1,
                                max_depth=5, random_state=SEED),
    X_tr, X_te, y_tr, y_te)
gb_imp = pd.Series(gb.feature_importances_, index=enc_features)

# 5. AdaBoost
evaluate_clf("AdaBoost",
    AdaBoostClassifier(n_estimators=100, random_state=SEED),
    X_tr, X_te, y_tr, y_te)

# 6-7. SVM
evaluate_clf("SVM (linear)",
    SVC(kernel="linear", random_state=SEED, probability=True, max_iter=5000),
    X_tr_s, X_te_s, y_tr, y_te)
evaluate_clf("SVM (rbf)",
    SVC(kernel="rbf", random_state=SEED, probability=True),
    X_tr_s, X_te_s, y_tr, y_te)

# 8. KNN – search for best k
k_scores = {}
for k in [3, 5, 7, 9, 11, 15]:
    tmp = KNeighborsClassifier(n_neighbors=k)
    tmp.fit(X_tr_s, y_tr)
    k_scores[k] = accuracy_score(y_te, tmp.predict(X_te_s))
best_k = max(k_scores, key=k_scores.get)
print(f"\n  KNN by k: {k_scores}")
print(f"  Best k = {best_k}\n")
evaluate_clf(f"KNN (k={best_k})",
    KNeighborsClassifier(n_neighbors=best_k),
    X_tr_s, X_te_s, y_tr, y_te)

# 9. Naive Bayes
evaluate_clf("Naive Bayes", GaussianNB(), X_tr_s, X_te_s, y_tr, y_te)

# 10. MLP Neural Network
evaluate_clf("MLP Neural Network",
    MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=500,
                  random_state=SEED, early_stopping=True),
    X_tr_s, X_te_s, y_tr, y_te)

print(f"\nAll {len(results)} encounter classifiers trained!")

Training Encounter-Type Classifiers
  Logistic Regression           Acc=0.6474  Prec=0.5739  Rec=0.6474  F1=0.5892
  Decision Tree                 Acc=0.8536  Prec=0.8505  Rec=0.8536  F1=0.8510
  Random Forest                 Acc=0.8853  Prec=0.8834  Rec=0.8853  F1=0.8811
  Gradient Boosting             Acc=0.8847  Prec=0.8835  Rec=0.8847  F1=0.8810
  AdaBoost                      Acc=0.6408  Prec=0.6007  Rec=0.6408  F1=0.5966
  SVM (linear)                  Acc=0.5673  Prec=0.5478  Rec=0.5673  F1=0.5408
  SVM (rbf)                     Acc=0.7634  Prec=0.7587  Rec=0.7634  F1=0.7496

  KNN by k: {3: 0.7962000358487185, 5: 0.7944075999283026, 7: 0.7920774332317619, 9: 0.7870586126545976, 11: 0.7800681125649758, 15: 0.7675210611220649}
  Best k = 3



## 5. Mortality Prediction (Binary Classification)

Predict patient mortality from aggregated healthcare-utilisation features.
Class imbalance handled with `class_weight='balanced'`.

In [ ]:
mort_features = [
    "Age","Gender_enc","Race_enc","Ethnicity_enc","Marital_enc",
    "total_encounters","total_cost","avg_cost","max_cost",
    "total_procs","avg_procs",
    "emergency_n","inpatient_n","ambulatory_n","wellness_n",
    "n_enc_types","total_coverage","avg_duration","reason_n",
    "Emergency_ratio","Cost_per_enc"]

X_m = pat[mort_features].values
y_m = pat["Is_Deceased"].values

X_m_tr, X_m_te, y_m_tr, y_m_te = train_test_split(
    X_m, y_m, test_size=0.2, random_state=SEED, stratify=y_m)
sc_m = StandardScaler()
X_m_tr_s = sc_m.fit_transform(X_m_tr)
X_m_te_s = sc_m.transform(X_m_te)

print(f"Mortality – Train: {len(y_m_tr)} (deceased {y_m_tr.sum()}, "
      f"{y_m_tr.mean()*100:.1f}%)")
print(f"Mortality – Test:  {len(y_m_te)} (deceased {y_m_te.sum()}, "
      f"{y_m_te.mean()*100:.1f}%)")
print("\nResults:")
print("=" * 90)

evaluate_clf("Mort – Logistic Reg.",
    LogisticRegression(max_iter=1000, random_state=SEED, class_weight="balanced"),
    X_m_tr_s, X_m_te_s, y_m_tr, y_m_te, store=mortality_results)

mort_rf, _ = evaluate_clf("Mort – Random Forest",
    RandomForestClassifier(n_estimators=200, random_state=SEED,
                           class_weight="balanced", n_jobs=-1),
    X_m_tr, X_m_te, y_m_tr, y_m_te, store=mortality_results)

evaluate_clf("Mort – Gradient Boosting",
    GradientBoostingClassifier(n_estimators=200, random_state=SEED),
    X_m_tr, X_m_te, y_m_tr, y_m_te, store=mortality_results)

evaluate_clf("Mort – SVM (rbf)",
    SVC(kernel="rbf", random_state=SEED, probability=True, class_weight="balanced"),
    X_m_tr_s, X_m_te_s, y_m_tr, y_m_te, store=mortality_results)

evaluate_clf("Mort – KNN",
    KNeighborsClassifier(n_neighbors=5),
    X_m_tr_s, X_m_te_s, y_m_tr, y_m_te, store=mortality_results)

evaluate_clf("Mort – MLP",
    MLPClassifier(hidden_layer_sizes=(64, 32), max_iter=500,
                  random_state=SEED, early_stopping=True),
    X_m_tr_s, X_m_te_s, y_m_tr, y_m_te, store=mortality_results)

# ---- ROC Curves ----
fig, ax = plt.subplots(figsize=(10, 8))
for name, data in mortality_results.items():
    model = data["model"]
    needs_scaled = any(k in name for k in ["Logistic","SVM","KNN","MLP"])
    X_roc = X_m_te_s if needs_scaled else X_m_te
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_roc)[:, 1]
    elif hasattr(model, "decision_function"):
        y_prob = model.decision_function(X_roc)
    else:
        continue
    fpr, tpr, _ = roc_curve(y_m_te, y_prob)
    short = name.replace("Mort – ", "")
    ax.plot(fpr, tpr, linewidth=2, label=f"{short} (AUC={auc(fpr,tpr):.3f})")

ax.plot([0,1],[0,1], "k--", linewidth=1, label="Random")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC Curves \u2013 Mortality Prediction", fontweight="bold")
ax.legend(loc="lower right")
fig.tight_layout()
save_plot(fig, "mortality_roc")

# ---- Mortality feature importance ----
mort_imp = pd.Series(mort_rf.feature_importances_, index=mort_features)
fig, ax = plt.subplots(figsize=(10, 7))
mort_imp.sort_values().plot(kind="barh", ax=ax, color="steelblue")
ax.set_title("Mortality \u2013 Feature Importance (Random Forest)", fontweight="bold")
ax.set_xlabel("Importance")
fig.tight_layout()
save_plot(fig, "mortality_feature_importance")

print("\nMortality prediction complete!")

## 6. Healthcare Cost Prediction (Regression)

Predict `TOTAL_CLAIM_COST` from encounter and patient features.

In [ ]:
reg_features = ["Age","GENDER_enc","RACE_enc","EncClass_enc",
                "Payer_enc","Hour","DayOfWeek","Month",
                "Has_reason","num_procedures","total_proc_cost","Duration_hours"]

X_r = enc[reg_features].values
y_r = enc["TOTAL_CLAIM_COST"].values
X_r_tr, X_r_te, y_r_tr, y_r_te = train_test_split(
    X_r, y_r, test_size=0.2, random_state=SEED)
sc_r = StandardScaler()
X_r_tr_s = sc_r.fit_transform(X_r_tr)
X_r_te_s = sc_r.transform(X_r_te)

def evaluate_reg(name, model, Xtr, Xte, ytr, yte):
    model.fit(Xtr, ytr)
    yp = model.predict(Xte)
    r2  = r2_score(yte, yp)
    mae = mean_absolute_error(yte, yp)
    rmse = np.sqrt(mean_squared_error(yte, yp))
    regression_results[name] = {"r2": r2, "mae": mae, "rmse": rmse,
                                "y_pred": yp, "model": model}
    print(f"  {name:30s}  R\u00b2={r2:.4f}  MAE={mae:.2f}  RMSE={rmse:.2f}")
    return model

print("Cost Prediction (Regression):")
print("=" * 80)
evaluate_reg("Linear Regression", LinearRegression(), X_r_tr_s, X_r_te_s, y_r_tr, y_r_te)
evaluate_reg("Ridge Regression", Ridge(alpha=1.0), X_r_tr_s, X_r_te_s, y_r_tr, y_r_te)
evaluate_reg("Lasso Regression", Lasso(alpha=1.0, max_iter=5000), X_r_tr_s, X_r_te_s, y_r_tr, y_r_te)
evaluate_reg("RF Regressor",
    RandomForestRegressor(n_estimators=200, max_depth=15, random_state=SEED, n_jobs=-1),
    X_r_tr, X_r_te, y_r_tr, y_r_te)
evaluate_reg("GB Regressor",
    GradientBoostingRegressor(n_estimators=200, learning_rate=0.1,
                               max_depth=5, random_state=SEED),
    X_r_tr, X_r_te, y_r_tr, y_r_te)

# ---- Actual vs Predicted ----
best_reg = max(regression_results, key=lambda k: regression_results[k]["r2"])
yp_best = regression_results[best_reg]["y_pred"]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].scatter(y_r_te, yp_best, alpha=0.15, s=6, color="steelblue")
lim = max(y_r_te.max(), yp_best.max())
axes[0].plot([0, lim], [0, lim], "r--", linewidth=1)
axes[0].set_xlabel("Actual Cost ($)"); axes[0].set_ylabel("Predicted Cost ($)")
axes[0].set_title(f"Actual vs Predicted \u2013 {best_reg}", fontweight="bold")

# R-squared comparison
reg_df = pd.DataFrame({n: {k: v for k, v in d.items() if k not in ("y_pred","model")}
                        for n, d in regression_results.items()}).T
reg_sorted = reg_df.sort_values("r2")
reg_sorted["r2"].plot(kind="barh", ax=axes[1], color="steelblue")
axes[1].set_title("Regression Model Comparison (R\u00b2)", fontweight="bold")
axes[1].set_xlabel("R\u00b2 Score")
fig.tight_layout()
save_plot(fig, "regression_results")

print(f"\nBest: {best_reg} (R\u00b2 = {regression_results[best_reg]['r2']:.4f})")
print("\n" + reg_df.round(4).to_string())

## 7. Patient Segmentation (Unsupervised Clustering)

Identify natural patient groups using K-Means, PCA, and DBSCAN.

In [ ]:
cluster_feats = ["Age","total_encounters","total_cost","avg_cost",
                  "total_procs","emergency_n","inpatient_n",
                  "Emergency_ratio","avg_duration","Cost_per_enc"]

X_cl = pat[cluster_feats].values
sc_cl = StandardScaler()
X_cl_s = sc_cl.fit_transform(X_cl)

# ---- Elbow + Silhouette ----
inertias, sils = [], []
K_range = range(2, 11)
for k in K_range:
    km = KMeans(n_clusters=k, random_state=SEED, n_init=10)
    lbl = km.fit_predict(X_cl_s)
    inertias.append(km.inertia_)
    sils.append(silhouette_score(X_cl_s, lbl))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
axes[0].plot(list(K_range), inertias, "bo-")
axes[0].set_title("Elbow Method", fontweight="bold")
axes[0].set_xlabel("k"); axes[0].set_ylabel("Inertia")
axes[1].plot(list(K_range), sils, "ro-")
axes[1].set_title("Silhouette Score", fontweight="bold")
axes[1].set_xlabel("k"); axes[1].set_ylabel("Score")
fig.tight_layout()
save_plot(fig, "elbow_silhouette")

best_k_cl = list(K_range)[int(np.argmax(sils))]
print(f"Best k = {best_k_cl}  (silhouette = {max(sils):.4f})")
km_final = KMeans(n_clusters=best_k_cl, random_state=SEED, n_init=10)
pat["Cluster"] = km_final.fit_predict(X_cl_s)

# ---- PCA visualisation ----
pca = PCA(n_components=2, random_state=SEED)
X_pca = pca.fit_transform(X_cl_s)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
sc1 = axes[0].scatter(X_pca[:,0], X_pca[:,1], c=pat["Cluster"],
                       cmap="Set2", s=30, alpha=0.7)
axes[0].set_title(f"K-Means (k={best_k_cl}) \u2013 PCA", fontweight="bold")
axes[0].set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)")
axes[0].set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)")
plt.colorbar(sc1, ax=axes[0], label="Cluster")

# ---- DBSCAN ----
dbscan = DBSCAN(eps=2.5, min_samples=5)
db_lbl = dbscan.fit_predict(X_cl_s)
n_db = len(set(db_lbl)) - (1 if -1 in db_lbl else 0)
sc2 = axes[1].scatter(X_pca[:,0], X_pca[:,1], c=db_lbl, cmap="Set1", s=30, alpha=0.7)
axes[1].set_title(f"DBSCAN ({n_db} clusters)", fontweight="bold")
axes[1].set_xlabel("PC1"); axes[1].set_ylabel("PC2")
plt.colorbar(sc2, ax=axes[1], label="Cluster")
fig.tight_layout()
save_plot(fig, "clustering_results")

print("\nCluster profiles (K-Means):")
profile = pat.groupby("Cluster")[cluster_feats].mean()
print(profile.round(2).to_string())

## 8. Hyperparameter Tuning

GridSearchCV (Random Forest) and RandomizedSearchCV (Gradient Boosting).

In [ ]:
print("GridSearchCV: Random Forest ...")
grid_rf = GridSearchCV(
    RandomForestClassifier(random_state=SEED, n_jobs=-1),
    {"n_estimators": [100, 200], "max_depth": [10, 15, 20],
     "min_samples_split": [2, 5]},
    cv=3, scoring="f1_weighted", n_jobs=-1, verbose=0)
grid_rf.fit(X_tr, y_tr)
print(f"  Best params: {grid_rf.best_params_}")
print(f"  Best CV F1:  {grid_rf.best_score_:.4f}")

yp_rf = grid_rf.predict(X_te)
results["RF (Tuned)"] = {
    "accuracy": accuracy_score(y_te, yp_rf),
    "precision": precision_score(y_te, yp_rf, average="weighted", zero_division=0),
    "recall": recall_score(y_te, yp_rf, average="weighted", zero_division=0),
    "f1_score": f1_score(y_te, yp_rf, average="weighted", zero_division=0),
    "y_pred": yp_rf, "model": grid_rf.best_estimator_}
print(f"  Test Acc:    {results['RF (Tuned)']['accuracy']:.4f}")

print("\nRandomizedSearchCV: Gradient Boosting ...")
rand_gb = RandomizedSearchCV(
    GradientBoostingClassifier(random_state=SEED),
    {"n_estimators": [100, 200, 300], "learning_rate": [0.01, 0.05, 0.1, 0.2],
     "max_depth": [3, 5, 7, 10], "min_samples_split": [2, 5, 10]},
    n_iter=20, cv=3, scoring="f1_weighted", random_state=SEED, n_jobs=-1, verbose=0)
rand_gb.fit(X_tr, y_tr)
print(f"  Best params: {rand_gb.best_params_}")
print(f"  Best CV F1:  {rand_gb.best_score_:.4f}")

yp_gb = rand_gb.predict(X_te)
results["GB (Tuned)"] = {
    "accuracy": accuracy_score(y_te, yp_gb),
    "precision": precision_score(y_te, yp_gb, average="weighted", zero_division=0),
    "recall": recall_score(y_te, yp_gb, average="weighted", zero_division=0),
    "f1_score": f1_score(y_te, yp_gb, average="weighted", zero_division=0),
    "y_pred": yp_gb, "model": rand_gb.best_estimator_}
print(f"  Test Acc:    {results['GB (Tuned)']['accuracy']:.4f}")
print("\nHyperparameter tuning complete!")

## 9. Cross-Validation & Advanced Evaluation

Stratified 5-fold CV, confusion matrices, feature importance, and learning curves.

In [ ]:
# ---- 5-Fold Cross-Validation ----
print("5-Fold Cross-Validation (Encounter Classification)")
print("=" * 70)
cv_models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=SEED),
    "Decision Tree": DecisionTreeClassifier(max_depth=15, random_state=SEED),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=SEED, n_jobs=-1),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, random_state=SEED),
    "Naive Bayes": GaussianNB(),
}
cv_data = []
for name, model in cv_models.items():
    acc_cv = cross_val_score(model, X_tr_s, y_tr, cv=5, scoring="accuracy")
    f1_cv  = cross_val_score(model, X_tr_s, y_tr, cv=5, scoring="f1_weighted")
    cv_data.append({"Model": name,
                    "Acc_mean": acc_cv.mean(), "Acc_std": acc_cv.std(),
                    "F1_mean": f1_cv.mean(), "F1_std": f1_cv.std()})
    print(f"  {name:25s}  Acc={acc_cv.mean():.4f}+/-{acc_cv.std():.4f}  "
          f"F1={f1_cv.mean():.4f}+/-{f1_cv.std():.4f}")
cv_df = pd.DataFrame(cv_data)

# CV bar chart
fig, ax = plt.subplots(figsize=(12, 5))
x = np.arange(len(cv_df)); w = 0.35
ax.bar(x-w/2, cv_df["Acc_mean"], w, yerr=cv_df["Acc_std"], capsize=3,
       label="Accuracy", color="steelblue")
ax.bar(x+w/2, cv_df["F1_mean"], w, yerr=cv_df["F1_std"], capsize=3,
       label="F1", color="coral")
ax.set_xticks(x); ax.set_xticklabels(cv_df["Model"], rotation=30, ha="right")
ax.set_title("Cross-Validation: Model Comparison", fontweight="bold")
ax.set_ylabel("Score"); ax.legend(); ax.set_ylim(0, 1.05)
fig.tight_layout()
save_plot(fig, "cv_comparison")

# ---- Feature Importance ----
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
for i, (name, imp) in enumerate([("Decision Tree", dt_imp),
                                  ("Random Forest", rf_imp),
                                  ("Gradient Boosting", gb_imp)]):
    imp.sort_values().plot(kind="barh", ax=axes[i], color="steelblue")
    axes[i].set_title(f"{name}\nFeature Importance", fontweight="bold")
    axes[i].set_xlabel("Importance")
fig.tight_layout()
save_plot(fig, "feature_importance")
print("\nFeature importance saved!")

# ---- Confusion Matrices ----
cm_models = {k: v for k, v in results.items()
             if "y_pred" in v and "Tuned" not in k}
n = len(cm_models); ncols = 3; nrows = (n + ncols - 1) // ncols
fig, axes = plt.subplots(nrows, ncols, figsize=(6*ncols, 5*nrows))
axes_f = axes.flatten()
for i, (name, data) in enumerate(cm_models.items()):
    cm = confusion_matrix(y_te, data["y_pred"])
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=axes_f[i],
                xticklabels=encounter_names, yticklabels=encounter_names)
    axes_f[i].set_title(f"{name}\nAcc={data['accuracy']:.3f}",
                        fontsize=10, fontweight="bold")
    axes_f[i].set_xlabel("Predicted"); axes_f[i].set_ylabel("Actual")
    axes_f[i].tick_params(axis="both", labelsize=7)
for i in range(n, len(axes_f)):
    axes_f[i].set_visible(False)
fig.tight_layout()
save_plot(fig, "confusion_matrices")
print("Confusion matrices saved!")

# ---- Learning Curves ----
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
lc_models = [
    ("Random Forest", RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1)),
    ("Gradient Boosting", GradientBoostingClassifier(n_estimators=100, random_state=SEED)),
    ("Logistic Regression", LogisticRegression(max_iter=1000, random_state=SEED)),
]
for i, (name, model) in enumerate(lc_models):
    sizes, tr_sc, va_sc = learning_curve(
        model, X_tr_s, y_tr, cv=5,
        train_sizes=np.linspace(0.1, 1.0, 8), scoring="accuracy", n_jobs=-1)
    axes[i].plot(sizes, tr_sc.mean(1), "o-", label="Train", color="steelblue")
    axes[i].fill_between(sizes, tr_sc.mean(1)-tr_sc.std(1),
                         tr_sc.mean(1)+tr_sc.std(1), alpha=0.1, color="steelblue")
    axes[i].plot(sizes, va_sc.mean(1), "o-", label="Validation", color="coral")
    axes[i].fill_between(sizes, va_sc.mean(1)-va_sc.std(1),
                         va_sc.mean(1)+va_sc.std(1), alpha=0.1, color="coral")
    axes[i].set_title(f"Learning Curve: {name}", fontweight="bold")
    axes[i].set_xlabel("Training Size"); axes[i].set_ylabel("Accuracy")
    axes[i].legend(loc="lower right"); axes[i].set_ylim(0, 1.05)
fig.tight_layout()
save_plot(fig, "learning_curves")
print("Learning curves saved!")

## 10. Ensemble Methods & Final Comparison

Voting and Stacking ensemble classifiers, plus a comprehensive model comparison.

In [ ]:
# Voting Classifier
print("Training Voting Classifier ...")
voting = VotingClassifier(estimators=[
    ("lr", LogisticRegression(max_iter=1000, random_state=SEED)),
    ("rf", RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1)),
    ("gb", GradientBoostingClassifier(n_estimators=100, random_state=SEED)),
], voting="soft", n_jobs=-1)
evaluate_clf("Voting Ensemble", voting, X_tr_s, X_te_s, y_tr, y_te)

# Stacking Classifier
print("\nTraining Stacking Classifier ...")
stacking = StackingClassifier(estimators=[
    ("rf", RandomForestClassifier(n_estimators=100, random_state=SEED, n_jobs=-1)),
    ("gb", GradientBoostingClassifier(n_estimators=100, random_state=SEED)),
    ("knn", KNeighborsClassifier(n_neighbors=best_k)),
], final_estimator=LogisticRegression(max_iter=1000, random_state=SEED),
   cv=3, n_jobs=-1)
evaluate_clf("Stacking Ensemble", stacking, X_tr_s, X_te_s, y_tr, y_te)

# ---- Final Model Comparison Table ----
rows = []
for name, data in results.items():
    rows.append({"Model": name, "Accuracy": data["accuracy"],
                 "Precision": data["precision"], "Recall": data["recall"],
                 "F1 Score": data["f1_score"]})
comp_df = pd.DataFrame(rows).sort_values("F1 Score", ascending=False)
print("\nFinal Model Comparison (Encounter Classification):")
print(comp_df.to_string(index=False))

# Comparison chart
fig, ax = plt.subplots(figsize=(14, 7))
comp_s = comp_df.sort_values("F1 Score")
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.9, len(comp_s)))
ax.barh(comp_s["Model"], comp_s["F1 Score"], color=colors)
ax.set_xlabel("F1 Score (Weighted)")
ax.set_title("All Models \u2013 F1 Score Comparison", fontweight="bold")
ax.set_xlim(0, 1.05)
for i, (_, row) in enumerate(comp_s.iterrows()):
    ax.text(row["F1 Score"]+0.005, i, f"{row['F1 Score']:.4f}", va="center", fontsize=9)
fig.tight_layout()
save_plot(fig, "model_comparison")

best_name = comp_df.iloc[0]["Model"]
best_f1   = comp_df.iloc[0]["F1 Score"]
print(f"\nBest model: {best_name} (F1 = {best_f1:.4f})")

## 11. HTML Report Generation

In [ ]:
# Build the HTML report using string concatenation
# (avoids f-string / CSS curly-brace conflicts)

css = """
<style>
* { margin:0; padding:0; box-sizing:border-box; }
body { font-family:'Segoe UI',system-ui,sans-serif; background:#f0f2f5; color:#333; line-height:1.6; }
.container { max-width:1200px; margin:0 auto; padding:20px; }
header { background:linear-gradient(135deg,#1a5276,#2e86c1); color:#fff; padding:40px; border-radius:12px; margin-bottom:30px; text-align:center; }
header h1 { font-size:2.2em; margin-bottom:10px; }
header p  { font-size:1.1em; opacity:.9; }
.card { background:#fff; border-radius:12px; padding:30px; margin-bottom:25px; box-shadow:0 2px 10px rgba(0,0,0,.08); }
.card h2 { color:#1a5276; border-bottom:3px solid #2e86c1; padding-bottom:10px; margin-bottom:20px; }
.card h3 { color:#2e86c1; margin:15px 0 10px; }
table { width:100%; border-collapse:collapse; margin:15px 0; }
th,td { padding:10px 14px; text-align:left; border-bottom:1px solid #eee; }
th { background:#f8f9fa; font-weight:600; color:#1a5276; }
tr:hover { background:#f8f9fa; }
.dashboard { display:grid; grid-template-columns:repeat(auto-fit,minmax(180px,1fr)); gap:15px; margin-bottom:25px; }
.stat-box { background:#fff; border-radius:10px; padding:20px; text-align:center; box-shadow:0 2px 8px rgba(0,0,0,.08); }
.stat-box .number { font-size:2em; font-weight:700; color:#2e86c1; }
.stat-box .label  { font-size:.9em; color:#666; margin-top:5px; }
.plot-img { max-width:100%; border-radius:8px; margin:15px 0; box-shadow:0 2px 8px rgba(0,0,0,.1); }
.explanation { background:#eaf2f8; border-left:4px solid #2e86c1; padding:15px; border-radius:0 8px 8px 0; margin:15px 0; }
.best { color:#27ae60; font-weight:700; }
footer { text-align:center; padding:30px; color:#999; font-size:.9em; }
</style>
"""

html = []
html.append("<!DOCTYPE html>")
html.append("<html lang='en'>")
html.append("<head><meta charset='UTF-8'>")
html.append("<meta name='viewport' content='width=device-width,initial-scale=1.0'>")
html.append("<title>Hospital Patient Records \u2013 ML Report</title>")
html.append(css)
html.append("</head><body><div class='container'>")

# --- Header ---
html.append("<header>")
html.append("<h1>\U0001f3e5 Hospital Patient Records \u2013 ML Analysis Report</h1>")
html.append("<p>Comprehensive Machine Learning Analysis of Synthea Healthcare Data</p>")
html.append(f"<p>{len(patients):,} patients &middot; {len(encounters):,} encounters &middot; {len(procedures):,} procedures</p>")
html.append("</header>")

# --- Dashboard ---
best_clf_name = comp_df.iloc[0]["Model"]
best_clf_f1   = comp_df.iloc[0]["F1 Score"]
best_reg_name = max(regression_results, key=lambda k: regression_results[k]["r2"])
best_reg_r2   = regression_results[best_reg_name]["r2"]
best_mort     = max(mortality_results, key=lambda k: mortality_results[k]["f1_score"])
best_mort_f1  = mortality_results[best_mort]["f1_score"]

html.append("<div class='dashboard'>")
for label, value in [
    ("Patients", f"{len(patients):,}"),
    ("Encounters", f"{len(encounters):,}"),
    ("ML Models Trained", str(len(results) + len(mortality_results) + len(regression_results))),
    ("Best Classifier F1", f"{best_clf_f1:.4f}"),
    ("Best Regression R\u00b2", f"{best_reg_r2:.4f}"),
    ("Plots Generated", str(len(report_images))),
]:
    html.append(f"<div class='stat-box'><div class='number'>{value}</div><div class='label'>{label}</div></div>")
html.append("</div>")

# --- Section 1: Dataset ---
html.append("<div class='card'><h2>\U0001f4cb 1. Dataset Overview</h2>")
html.append("<p>This analysis uses <strong>Synthea</strong>-generated synthetic hospital records "
            "from Massachusetts General Hospital, Boston, MA.</p>")
html.append("<table><tr><th>Table</th><th>Rows</th><th>Columns</th><th>Description</th></tr>")
for tname, tdf, tdesc in [
    ("patients", patients, "Demographics, birth/death dates, location"),
    ("encounters", encounters, "Hospital visits, costs, payer, diagnosis"),
    ("procedures", procedures, "Medical procedures linked to encounters"),
    ("payers", payers, "Insurance providers"),
    ("organizations", organizations, "Hospital information"),
]:
    html.append(f"<tr><td><strong>{tname}</strong></td><td>{len(tdf):,}</td>"
                f"<td>{tdf.shape[1]}</td><td>{tdesc}</td></tr>")
html.append("</table>")
html.append("<div class='explanation'><strong>Key insight:</strong> "
            "All encounters occur at a single hospital. The dataset spans 2011\u20132022 "
            "with 10 insurance payers (including uninsured).</div>")
html.append("</div>")

# --- Section 2: EDA ---
html.append("<div class='card'><h2>\U0001f4ca 2. Exploratory Data Analysis</h2>")
for name, caption in [
    ("overview_dashboard", "Overview: encounter types, demographics, costs, and mortality age distributions"),
    ("encounters_timeline", "Monthly encounter volume across the 11-year period"),
    ("correlation_heatmap", "Pearson correlation between numeric features"),
    ("payer_distribution", "Encounters broken down by insurance payer"),
    ("top_procedures", "The 15 most frequently performed medical procedures"),
    ("gender_encounter_heatmap", "Encounter-type proportions by patient gender"),
]:
    if name in report_images:
        html.append(f"<h3>{caption}</h3>")
        html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images[name]}'>")
html.append("</div>")

# --- Section 3: Encounter Classification ---
html.append("<div class='card'><h2>\U0001f916 3. Encounter-Type Classification</h2>")
html.append("<p>Predicting encounter type (ambulatory, emergency, inpatient, outpatient, "
            "urgentcare, wellness) from patient demographics, temporal features, and procedure data.</p>")
html.append("<table><tr><th>Model</th><th>Accuracy</th><th>Precision</th><th>Recall</th><th>F1 Score</th></tr>")
for _, row in comp_df.iterrows():
    best_cls = " class='best'" if row["Model"] == best_clf_name else ""
    html.append(f"<tr{best_cls}><td>{row['Model']}</td>"
                f"<td>{row['Accuracy']:.4f}</td><td>{row['Precision']:.4f}</td>"
                f"<td>{row['Recall']:.4f}</td><td>{row['F1 Score']:.4f}</td></tr>")
html.append("</table>")
if "confusion_matrices" in report_images:
    html.append("<h3>Confusion Matrices</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['confusion_matrices']}'>")
if "model_comparison" in report_images:
    html.append("<h3>Model Comparison</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['model_comparison']}'>")
html.append("</div>")

# --- Section 4: Mortality ---
html.append("<div class='card'><h2>\u2695\ufe0f 4. Mortality Prediction</h2>")
html.append("<p>Binary classification predicting patient mortality from aggregated "
            "healthcare utilisation features. Class imbalance handled with balanced class weights.</p>")
html.append("<table><tr><th>Model</th><th>Accuracy</th><th>Precision</th><th>Recall</th><th>F1 Score</th></tr>")
for name, data in sorted(mortality_results.items(), key=lambda x: -x[1]["f1_score"]):
    best_cls = " class='best'" if name == best_mort else ""
    html.append(f"<tr{best_cls}><td>{name}</td>"
                f"<td>{data['accuracy']:.4f}</td><td>{data['precision']:.4f}</td>"
                f"<td>{data['recall']:.4f}</td><td>{data['f1_score']:.4f}</td></tr>")
html.append("</table>")
if "mortality_roc" in report_images:
    html.append("<h3>ROC Curves</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['mortality_roc']}'>")
if "mortality_feature_importance" in report_images:
    html.append("<h3>Feature Importance</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['mortality_feature_importance']}'>")
html.append("</div>")

# --- Section 5: Regression ---
html.append("<div class='card'><h2>\U0001f4b0 5. Cost Prediction (Regression)</h2>")
html.append("<p>Predicting TOTAL_CLAIM_COST from encounter and patient features.</p>")
html.append("<table><tr><th>Model</th><th>R\u00b2</th><th>MAE ($)</th><th>RMSE ($)</th></tr>")
for name in sorted(regression_results, key=lambda k: -regression_results[k]["r2"]):
    d = regression_results[name]
    best_cls = " class='best'" if name == best_reg_name else ""
    html.append(f"<tr{best_cls}><td>{name}</td>"
                f"<td>{d['r2']:.4f}</td><td>{d['mae']:.2f}</td><td>{d['rmse']:.2f}</td></tr>")
html.append("</table>")
if "regression_results" in report_images:
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['regression_results']}'>")
html.append("</div>")

# --- Section 6: Clustering ---
html.append("<div class='card'><h2>\U0001f9e9 6. Patient Segmentation</h2>")
html.append(f"<p>K-Means clustering identified <strong>{best_k_cl} patient segments</strong> "
            f"(silhouette = {max(sils):.4f}). DBSCAN was used for density-based comparison.</p>")
if "elbow_silhouette" in report_images:
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['elbow_silhouette']}'>")
if "clustering_results" in report_images:
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['clustering_results']}'>")
html.append("</div>")

# --- Section 7: CV & Learning Curves ---
html.append("<div class='card'><h2>\U0001f4c8 7. Cross-Validation & Learning Curves</h2>")
if "cv_comparison" in report_images:
    html.append("<h3>5-Fold Cross-Validation</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['cv_comparison']}'>")
if "feature_importance" in report_images:
    html.append("<h3>Feature Importance (3 tree-based models)</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['feature_importance']}'>")
if "learning_curves" in report_images:
    html.append("<h3>Learning Curves</h3>")
    html.append(f"<img class='plot-img' src='data:image/png;base64,{report_images['learning_curves']}'>")
html.append("</div>")

# --- Section 8: Conclusions ---
html.append("<div class='card'><h2>\U0001f3af 8. Key Findings & Conclusions</h2>")
html.append("<div class='explanation'>")
html.append("<h3>Encounter Classification</h3>")
html.append(f"<p>The best classifier was <strong>{best_clf_name}</strong> with F1 = {best_clf_f1:.4f}. "
            "Tree-based and ensemble methods consistently outperformed linear models. "
            "The most important features were procedure count and total procedure cost, "
            "confirming that the type of care delivered strongly signals the encounter class.</p>")
html.append("<h3>Mortality Prediction</h3>")
html.append(f"<p>The best mortality predictor was <strong>{best_mort.replace('Mort \u2013 ','')}</strong> "
            f"(F1 = {best_mort_f1:.4f}). Age and total healthcare cost emerged as the strongest "
            "predictors. Using balanced class weights was essential due to the low mortality rate.</p>")
html.append("<h3>Cost Prediction</h3>")
html.append(f"<p><strong>{best_reg_name}</strong> achieved R\u00b2 = {best_reg_r2:.4f}. "
            "Procedure cost and duration were the dominant predictors, as expected in a "
            "fee-for-service cost model.</p>")
html.append("<h3>Patient Segmentation</h3>")
html.append(f"<p>K-Means identified {best_k_cl} distinct patient segments. Clusters differ primarily "
            "in healthcare utilisation intensity (total encounters, emergency visits, costs). "
            "These segments can support targeted intervention strategies.</p>")
html.append("</div></div>")

# --- Footer ---
html.append("<footer>")
html.append("<p>Generated with scikit-learn, pandas, matplotlib, seaborn</p>")
html.append("<p>Dataset: Synthea \u2013 Walonoski et al., JAMIA 2018</p>")
html.append("</footer>")
html.append("</div></body></html>")

# ---- Write ----
report_path = "outputs/hospital_ml_report.html"
with open(report_path, "w", encoding="utf-8") as f:
    f.write("\n".join(html))

size_kb = os.path.getsize(report_path) / 1024
print(f"\u2705 HTML Report generated: {report_path}")
print(f"   File size: {size_kb:.1f} KB")
print(f"   Embedded images: {len(report_images)}")
print(f"\n\U0001f389 Analysis complete! Open the HTML file in a browser to view the full report.")